## Симуляция секвенирования

Первое задание заключается в том, чтобы написать симуляцию секвенирования ридов при помощи illumina. Для этого необходимо сгенерировать случайно строку длинной 50 000 символов, над алфавитом {A, T, G, C}. После этого случайно выбирать подстроки, длинна которых распределена нормально со средним 250 и среднеквадратическим отклонением 30, каждую такую подстроку "секвенировать".</br>
Под словом "секвенировать" в данном случае имеется ввиду симуляция того процесса, который мы обсуждали на лекции. Вы считываете очередной символ подстроки, например, 100 раз, но в N случаях из 100 ошибаетесь и считываете любой нуклеотид, кроме правильного с одинаковой вероятностью. N принадлежит равномерному распределению от 0 до 75. По получившемуся набору вычисляете наибоее вероятный нуклеотид (тот, которого больше всего) и его качество прочтения, чтобы записать его в формате FASTQ.</br>
При симуляции важно запоминать ID рида и для каждого ID позиции, в которых нуклеотид был считан неверно и какой должен быть на самом деле (для этого просто можно хранить 2 числа и помнить что у вас есть исходная строка, откуда берутся риды). Это понадобится для выполнения следующих заданий.</br>
Всего ридов пусть будет 50К.</br>

In [1]:
import numpy as np
import pandas as pd

np.random.seed(0)

nucleotides = ['A', 'T', 'G', 'C']

sequence_string = ''.join(np.random.choice(nucleotides, 50000).tolist())
read_mistakes = {'read_number': [], 'start_position_ID': [], 'mistake_position_ID': []}

for i in range(1, 50001):
    read_mistakes['read_number'].append(i)
    read_len = int(np.random.normal(250, 30, 1)[0])
    read_start = np.random.default_rng().integers(0, 50000 - read_len)
    read_mistakes['start_position_ID'].append(read_start)
    read_str = sequence_string[read_start:read_start+read_len]
    mistakes_str = ''
    mistakes_pos = []
    for nucleotide_idx in range(read_len):
        nucleotide = read_str[nucleotide_idx]
        n = np.random.default_rng().integers(1, 75)
        mistakes = np.random.choice([nuc for nuc in nucleotides if nuc != nucleotide], n)
        vals, counts = np.unique(mistakes, return_counts=True)
        if max(counts) > 100-n:
            read_str = read_str[:nucleotide_idx] + vals[np.argmax(counts)] + read_str[nucleotide_idx+1:]
            q = int(-10*np.log10((100-max(counts))/100))
            mistakes_pos.append(nucleotide_idx)
        else:
            q = int(-10*np.log10(n/100))
        mistakes_str += chr(q + 33)
    read_mistakes['mistake_position_ID'].append(mistakes_pos)
    mistakes_str += '\n'    
    with open(f"reads.fasta", "a") as file:
        file.write(f'@READ_SEQ_{i}\n')
        file.write(read_str + '\n')
        file.write('+\n')
        file.write(mistakes_str)

pd.DataFrame.from_dict(read_mistakes).to_csv('read_mistakes.csv', index=False)

with open("sequense.fasta", "w") as file:
    file.write('@MAIN_SEQ_0\n')
    file.write(sequence_string)



## Удаление ошибок (Trimmomatic) 
Получившиеся риды обработайте Trimmomatic со стандартными параметрами с лекции. Пользуясь тем, что вы знаете, для каждого рида ground truth, посчитайте, в скольких случаях Trimmomatic удалил нуклеотиды, которые были считаны верно, а в скольких действительно удалил ошибочные.

In [1]:
! trimmomatic SE -phred33 reads.fasta reads_trim.fasta ILLUMINACLIP:TruSeq3-SE.fa:2:30:10:2:True LEADING:3 TRAILING:3 MINLEN:36

TrimmomaticSE: Started with arguments:
 -phred33 reads.fasta reads_trim.fasta ILLUMINACLIP:TruSeq3-SE.fa:2:30:10:2:True LEADING:3 TRAILING:3 MINLEN:36
Automatically using 4 threads
Using Long Clipping Sequence: 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTA'
Using Long Clipping Sequence: 'AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC'
ILLUMINACLIP: Using 0 prefix pairs, 2 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Input Reads: 50000 Surviving: 50000 (100.00%) Dropped: 0 (0.00%)
TrimmomaticSE: Completed successfully


In [2]:
import pandas as pd

df_mistakes = pd.read_csv('/home/pk/Desktop/BOTAY/BioinformaticsCourse2024PK/homework/2_1/read_mistakes.csv')

with open("reads_trim.fasta", "r") as file1:
    lines_trimm = file1.readlines()

with open("reads.fasta", "r") as file2:
    lines = file2.readlines()

In [3]:
true_del = 0
false_del = 0
id = 1

for i in range(0, len(lines_trimm), 4):
    try:
        mistakes_idxs = set(map(int, df_mistakes.loc[df_mistakes['read_number'] == id]['mistake_position_ID'].iloc[0][1:-1].split(', ')))
    except:
        mistakes_idxs = []  # если пустой массив
    read = lines[i+1]
    read_trimm = lines_trimm[i+1] + '$'
    for elem_idx in range(len(read)):
        if read[elem_idx] != read_trimm[elem_idx]:
            read_trimm = read_trimm[:elem_idx] + '-' + read_trimm[elem_idx:]
            if elem_idx in mistakes_idxs:
                true_del += 1
            else:
                false_del += 1
    id += 1




In [4]:
print('Верные удаления:', true_del)
print('Ошибочные удаления:', false_del)

Верные удаления: 3942
Ошибочные удаления: 43724


## Коррекция ошибок 

Получившиеся риды обработайте одним(любым) из инструментов, которые использованы и проанализированны в этой [статье](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-020-01988-3). Посчитайте TP, FP, TN, FN.

Для исправления ошибок я решил использовать метод [Lighter](https://github.com/mourisl/Lighter). Размер k-мера был выбран равным 20 

In [2]:
! conda install lighter --channel bioconda

Channels:
 - bioconda
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [1]:
! lighter -r reads.fasta -K 20 50000 -t 10

[2024-06-02 11:57:37] =============Start====================
[2024-06-02 11:57:37] Scanning the input files to infer alpha(sampling rate)
[2024-06-02 11:57:37] Average coverage is 249.634 and alpha is 0.028
[2024-06-02 11:57:37] Bad quality threshold is "!"
[2024-06-02 11:57:39] Finish sampling kmers
[2024-06-02 11:57:39] Bloom filter A's false positive rate: 0.335902
[2024-06-02 11:57:39] The error rate is high. Lighter adjusts -maxcor to 5 and bad quality threshold to """.
[2024-06-02 11:57:40] Finish storing trusted kmers
[2024-06-02 11:57:40] Finish error correction
Processed 50000 reads:
	74 are error-free
	Corrected 343435 bases(6.878881 corrections for reads with errors)
	Trimmed 0 reads with average trimmed bases 0.000000
	Discard 0 reads


In [4]:
import pandas as pd

df_mistakes = pd.read_csv('/home/pk/Desktop/BOTAY/BioinformaticsCourse2024PK/homework/2_1/read_mistakes.csv')

with open("reads.cor.fq", "r") as file1:
    lines_cor = file1.readlines()

with open("reads.fasta", "r") as file2:
    lines = file2.readlines()

with open("sequense.fasta", "r") as file3:
    sequence = file3.readlines()[1]

In [7]:
TP = 0  # ошибочные нуклеотиды, которые были исправлены
TN = 0  # правильные нуклеотилы, которые были нетронуты
FP = 0  # правильные нуклеодиты, которые были исправлены
FN = 0  # ошибочные нуклеотиды, которые не были исправлены
id = 1

for i in range(0, len(lines_cor), 4):
    start_position = int(df_mistakes.loc[df_mistakes['read_number'] == id]['start_position_ID'].iloc[0])
    try:
        mistakes_idxs = set(map(int, df_mistakes.loc[df_mistakes['read_number'] == id]['mistake_position_ID'].iloc[0][1:-1].split(', ')))
    except:
        mistakes_idxs = []
    read = lines[i+1]
    read_cor = lines_cor[i+1]
    for elem_idx in range(len(read)):
        if read[elem_idx] != read_cor[elem_idx]:
            if elem_idx in mistakes_idxs and read_cor[elem_idx] == sequence[start_position+elem_idx]:
                TP += 1
            elif elem_idx in mistakes_idxs and read_cor[elem_idx] != sequence[start_position+elem_idx]:
                FN += 1
            elif elem_idx not in mistakes_idxs:
                FP += 1
        else:
            if elem_idx in mistakes_idxs:
                FN += 1
            else:
                TN += 1
    id += 1

print('TP =', TP)
print('FP =', FP)
print('TN =', TN)
print('FN =', FN)

TP = 343432
FP = 3
TN = 12150054
FN = 38217


## Сделайте какой-то вывод

Лучше использовать алгоритмы по исправлению ридов с ошибками, так как они позволяют более точно и без удаления нуклеотидов (и, следовательно, потери информации) улучшать качество получаемых в ходе секвенирования последовательностей